# Compare building height and population

1. Stack population and builidng heights
2. Create 2o population layer describing 3x3 population sum
3. Calculate service access

TODO: Add metadata

1. DLR height
2. WP Population


In [4]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../../GOST/")

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.misc as misc
import GOSTRocks.osmMisc as osm_misc

import hot_spot_mapping as hot

In [7]:
wp_vrt = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/WP_2020_100m.vrt"
wp = rasterio.open(wp_vrt)
globcover_data = "/home/public/Data/GLOBAL/LANDCOVER/GLOBCOVER/2015/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
inWSF_folder = "/home/public/Data/PROJECTS/LEI/DLR_V2"
out_folder = "/home/wb411133/data/Projects/HeightComparison"

#Get a list of cities
allCities = {}
for root, dirs, files in os.walk(inWSF_folder):
    for f in files:
        if f[-4:] == ".tif":
            cityName = f.split("_")[0]
            if not cityName in allCities.keys():
                allCities[cityName] = [os.path.join(root, f)]
            else:
                allCities[cityName].append(os.path.join(root, f))

In [8]:
pop_raster = rasterio.open(wp_vrt)
globR = rasterio.open(globcover_data)


In [ ]:
importlib.reload(hot)
bad_cities = []
for city_name, rasters in allCities.items():
    try:
        print(city_name)
        cur_folder = os.path.join(out_folder, city_name)
        if not os.path.exists(cur_folder):
            os.makedirs(cur_folder)
        city = hot.city_hotspot(rasters[0], cur_folder)
        city.extract_other_rasters(pop_raster, globR)
        rosads = city.extract_osm_data()
        xx = city.generate_walking_raster()
        city.calculate_accessibility()
        city.calculate_pop_hotspots()
    except:
        bad_cities.append(city_name)

Zwolle
Aalborg
Abadan
AbuZaby
accra.tif
Accra
Adler
Agri
Ahmadabad
Ahvaz
Akita
Aksaray
alexandria.tif
Alexandria
Algiers
Aligudarz
Alikakh
Al-Marj
AlorSetar
Alwar
Amirabad
Amloricolliery
Anand
Anning
Anqing
Antwerpen
Arusha
As-Sinbillawayn
Astrakhan
Attur
Auckland
Aurangabad
Bacolod
Bafoussam
Baghdad
Baku
Balikesir
Bamako
Bangkok
Baora
Baragaon
Barretos
Barshi
Beijing
Beipiao
Beira
BelaMuchhpakauni
Belgrade
BeloHorizonte
Berezniki
Berlin
Error writing roads to disk
Bilimora
Bingxi
Bishan
Bogota
Bojonegoro
BojongGede
Bombay
BouSaada
BragancaPaulista
Budapest
BuenosAires
Bukhoro
Burnley
Busan
Cabimas
CaboSanLucas
Cagliari
Cairo
Calicut
CaoLanh
CaoMat
Caracas
Caserta
CebuCity
Chakdaha
Chalus
Owo
Changzhi
Changzhou
Chengdu
Chengguan
Cheonan
Chicago
Chilmari
Chinhae
Chinju
Chittaurgarh
Christchurch
Chucand
Cirebon
Cizre
Clermont-Ferran
Cleveland
Cochabamba
Coimbatore
Conghua
Cordoba
Cuauhtemoc
Culiacan
Curitiba
Dadu
Danjiangkou
DeirEl-Zor
Dengfeng
Dhaka
Dhupguri
Dila
Diourbel
Divinopolis
Du